<a href="https://colab.research.google.com/github/Me-RockingAbhi/My-Projects/blob/master/hacker-earth_drugs_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
data_tr=pd.read_csv("/content/gdrive/My Drive/drug_train.csv")
data_te=pd.read_csv("/content/gdrive/My Drive/drug_test.csv")


In [0]:
data_tr.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [0]:
print(data_tr.isnull().sum())
print(data_te.isnull().sum())


patient_id                    0
name_of_drug                  0
use_case_for_drug             0
review_by_patient             0
effectiveness_rating          0
drug_approved_by_UIC          0
number_of_times_prescribed    0
base_score                    0
dtype: int64
patient_id                    0
name_of_drug                  0
review_by_patient             0
drug_approved_by_UIC          0
number_of_times_prescribed    0
use_case_for_drug             0
effectiveness_rating          0
dtype: int64


In [0]:
print(data_tr.dtypes)
print(data_tr.shape)
print(data_te.shape)
ytrain=data_tr['base_score']
data_tr=data_tr.drop('base_score',axis=1)
print(data_tr.shape)
data_tr.head()

patient_id                      int64
name_of_drug                   object
use_case_for_drug              object
review_by_patient              object
effectiveness_rating            int64
drug_approved_by_UIC           object
number_of_times_prescribed      int64
base_score                    float64
dtype: object
(32165, 8)
(10760, 7)
(32165, 7)


,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43


In [0]:
data3=data_tr.append(data_te)
data3=data3.drop('patient_id',axis=1)
data3.head()

,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43


In [0]:
print(data3['name_of_drug'].nunique())
print(data3['use_case_for_drug'].nunique())

2381
670


In [0]:
data=pd.read_csv("/content/gdrive/My Drive/movie_rev_Train.csv")
xtr=data['review']
ytr=data['label']
xte=data3['review_by_patient']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
ytr=le.fit_transform(ytr)
print(ytr)

[1 1 1 ... 0 1 1]


In [0]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
def get_stemmed_reviw(review):
    review=review.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    token=tokenizer.tokenize(review)
    en_stopwars=set(stopwords.words('english'))
    new_token=[token for token in token if token not in en_stopwars]
    ps=PorterStemmer()
    stem_tokn=[ps.stem(token)for token in new_token]
    clean_review=' '.join(stem_tokn)
    return(clean_review)
p=[get_stemmed_reviw(i) for i in xtr]
q=[get_stemmed_reviw(i) for i in xte]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Activation,Dropout,Dense
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D,LSTM,GlobalMaxPooling1D,SimpleRNN,Flatten,LSTM,Bidirectional,MaxPool1D
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(p)
tokenizer.fit_on_texts(q)
p= tokenizer.texts_to_sequences(p)
q= tokenizer.texts_to_sequences(q)

In [0]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
path='/content/gdrive/My Drive/GoogleNews-vectors-negative300.bin.gz'
maxlen=300
p = pad_sequences(p, padding='post', maxlen=maxlen)
q = pad_sequences(q, padding='post', maxlen=maxlen)
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True, limit=100000)
embedding_weights = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    #embedding_vector = word2vec.get(word)
    try:
        embedding_weights[index] = word2vec[word]
    except:
        pass 

76002


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from keras.callbacks import EarlyStopping 
from keras.regularizers import l2
model=Sequential()
em_layer=Embedding(76002,300,weights=[embedding_weights],input_length=maxlen,trainable=False)
model.add(em_layer)
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(72587,300)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 300)          22800600  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 298, 256)          230656    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 23,031,513
Trainable params: 230,913
Non-trainable params: 22,800,600
_________________________________________________________________


In [0]:
history = model.fit(p, ytr, batch_size=60, epochs=4, verbose=1, validation_split=0.2)
ans=model.predict_classes(q)

Train on 32000 samples, validate on 8000 samples
Epoch 1/4
32000/32000 [==============================] - 196s 6ms/step - loss: 0.4653 - accuracy: 0.7717 - val_loss: 0.3679 - val_accuracy: 0.8385
Epoch 2/4
32000/32000 [==============================] - 195s 6ms/step - loss: 0.3626 - accuracy: 0.8418 - val_loss: 0.3349 - val_accuracy: 0.8534
Epoch 3/4
32000/32000 [==============================] - 197s 6ms/step - loss: 0.3241 - accuracy: 0.8588 - val_loss: 0.3226 - val_accuracy: 0.8596
Epoch 4/4
32000/32000 [==============================] - 194s 6ms/step - loss: 0.2986 - accuracy: 0.8728 - val_loss: 0.3148 - val_accuracy: 0.8637
